In [ ]:
import requests
from bs4 import BeautifulSoup
from time import sleep 
from random import random


import json
import urllib.request
base_url = 'http://www.portaldatransparencia.gov.br/'

def read_table(table):
    rows = table.findAll("tr")
    header = rows[0]
    #print ('header')
    cols = header.findAll('th')
    #print ('# of cols' + str(len(cols)))

    my_json = []
    
    #for col in cols:
        #print (col.getText().strip())

    for row in range(1,len(rows)):
        my_dict = {}
        
        a = 'row ' + str(row)
        cols = rows[row].findAll("td", {"class":"firstChild"})
        cols_valor = rows[row].findAll("td", {"class":"colunaValor"})
        for col in cols:
            for col2 in cols_valor:
                b = col2.getText().strip()
            c = col.getText().strip()
        
        
        mes_ano = c.split('/')
        my_dict['Mes'] = mes_ano[0]
        my_dict['Ano'] = int(mes_ano[1])
        my_dict['Valor'] = float(b.replace(".", "").replace(",00", ".00"))
        my_json.append(my_dict)
    
    return my_json
   
    
    
    
def read_person(person_url):
    print ('read_person')
    print (person_url)
    person = urllib.request.urlopen(person_url)
    person_soup = BeautifulSoup(person, "lxml")

    tables = person_soup.findAll("table")
    g = read_table(tables[1])
    #print (json.dumps(g, indent=2, sort_keys=True, ensure_ascii=False))
    return g
    
    
#read_person('http://www.portaldatransparencia.gov.br/PortalTransparenciaPesquisaAcaoDetalhado.asp?Exercicio=2016&textoPesquisa=&textoPesquisaAcao=&codigoAcao=8442&codigoFuncao=08&siglaEstado=AC&codigoMunicipio=0643&codFavorecido=6296326&cpfcnpjnis=00016167611395')

#read_person('http://www.portaldatransparencia.gov.br/PortalTransparenciaPesquisaAcaoDetalhado.asp?Exercicio=2016&textoPesquisa=&textoPesquisaAcao=&codigoAcao=8442&codigoFuncao=08&siglaEstado=AC&codigoMunicipio=0157&codFavorecido=4284887&cpfcnpjnis=00021230735463')


In [ ]:
from pymongo import MongoClient
import json
def read_json(estado,municipio,pagina,nis):
    cliente = MongoClient('localhost', 27017)
    banco = cliente.projeto_final
    projeto_transparencia = banco.test_collection
    users = banco.projeto_transparencia
    vazio = users.find({}).count()
    if vazio != 0: #pro caso do banco estar vazio 
        a = users.find_one({"NIS": nis ,"Município" : municipio  })
        json_pause = {}
        if a is None :
            json_pause['Pagina'] = int(pagina)
            json_pause['Estado'] = estado
            json_pause['Municipio'] = municipio
            json_pause['Nis'] = nis
            with open('teste_acrelandia.json', 'w') as arquivo:
                json.dump(json_pause, arquivo, indent=4, sort_keys=True)
            return True
        else:
            return False
    else:
        return True
    
def verifica_pagina():
    with open('teste_acrelandia.json', 'r') as arquivo:
        json_pause = json.loads(arquivo.read())
    return json_pause

def verifica_municipio(estado,municipio):
    with open('teste_acrelandia.json', 'r') as arquivo:
        json_pauses = json.loads(arquivo.read())
    #if estado is not None:
        if json_pauses["Municipio"] != municipio:
            json_pause= {}
            json_pause['Pagina'] = 1
            json_pause['Estado'] = estado
            json_pause['Municipio'] = municipio
            json_pause['Nis'] = ""
            #json_pause['Lido_Municipio'] = json_pauses["Municipio"]
            with open('teste_acrelandia.json', 'w') as arquivo:
                json.dump(json_pause, arquivo, indent=4, sort_keys=True)
            return json_pause
        else:
            return json_pauses
    #else: 
        #g =[]     
        #f = municipio
        #g.append(f)
        #return g 
    


        

In [ ]:
import os
def read_city(city_url):
    print (city_url)
    b = []
    c = []
    d = {}
    city_page = urllib.request.urlopen(city_url)
    city_page_soup = BeautifulSoup(city_page, "lxml")
    y = city_page_soup.findAll("tr")
    w = y[3]
    e = w.findAll('td')
    f = e[0]
    g = f.find('b')
    estado = g.getText()
    print (estado)
    k = y[6]
    t = k.findAll('td')
    l = t[0]
    #q = l.getText()
    q = l.find('b')
    municipio = q.getText()
    print (municipio)
    city_pagination = city_page_soup.find(id='paginacao').p.getText()
    print(city_pagination)
    city_pages = str.split(city_pagination, '/')[1]
    #print (city_pages) #total de páginas
    npages = int(city_pages)+1
    #with open('teste_acrelandia.json', 'w') as arquivo:
    cont = 0
    for city_page in range(1,npages):
        json_pause = verifica_pagina()
        json_municipio = verifica_municipio(estado,municipio)
        if json_pause["Pagina"] != json_municipio["Pagina"]:
            pagina = json_municipio["Pagina"]
        else:
            pagina = json_pause["Pagina"]
        if pagina <= city_page:  
            city_page_url = city_url + '&Pagina=' + str(city_page)
            pagina = '\nPAGINA ' + str(city_page)
            print (pagina)
            a = read_city_page(city_page_url,municipio,estado,city_page)
                #d['pagina '+ str(city_page)] = a
                    #sleep(1+random())
        else:
            print ("Página " + str(city_page) + " já lida")
            cont = cont + 1
            print (cont)
                    #d["total_de_paginas"] = city_page
                    #json.dump(d, arquivo, indent=4, sort_keys=True)
             
            
#read_city('http://www.portaldatransparencia.gov.br/PortalTransparenciaPesquisaAcaoFavorecido.asp?Exercicio=2016&textoPesquisa=&textoPesquisaAcao=&codigoAcao=8442&codigoFuncao=08&siglaEstado=AC&codigoMunicipio=0643')
#read_city('http://www.portaldatransparencia.gov.br/PortalTransparenciaPesquisaAcaoFavorecido.asp?Exercicio=2016&textoPesquisa=&textoPesquisaAcao=&codigoAcao=8442&codigoFuncao=08&siglaEstado=AC&codigoMunicipio=0157')
#read_city('http://www.portaldatransparencia.gov.br/PortalTransparenciaPesquisaAcaoFavorecido.asp?Exercicio=2016&textoPesquisa=&textoPesquisaAcao=&codigoAcao=8442&codigoFuncao=08&siglaEstado=AC&codigoMunicipio=0105')

In [ ]:
import urllib.request
from bs4 import BeautifulSoup
from pymongo import MongoClient
import datetime
import json


def read_city_page(city_page_url,municipio,estado,pagina):
    cliente = MongoClient('localhost', 27017)

    banco = cliente.projeto_final
    projeto_transparencia = banco.test_collection
    print ('read_city_page')
    print (city_page_url)
    city_page = urllib.request.urlopen(city_page_url)
    city_page_soup = BeautifulSoup(city_page, "lxml", from_encoding='utf-8')

    tables = city_page_soup.findAll("table")
    rows = tables[1].findAll("tr")
    header = rows[0]
    #print ('header')
    lista = []
    json_pause = {}

    nrows = len(rows)
    for row in range(1,nrows):
        linha = {}
        cols = rows[row].findAll('td')
        nis = cols[0].getText().strip()
        nome = cols[1].getText().strip()
        total = cols[2].getText().strip()
        print (nome + ' - ' + nis + ' - ' + total)
        person_url = base_url + cols[1].find('a').get('href')
        valorTotal = cols[2].getText().strip()
        js = read_person(person_url)
        linha['Estado'] = estado
        linha['Município'] = municipio
        linha['Valores'] = js
        linha['Nome'] = nome
        linha['NIS'] = nis
        linha['Total'] = float(total.replace(".","").replace(",00",""))
        t = linha
        verifica = read_json(estado,municipio,pagina,nis)
        if verifica:
            projeto_transparencia = banco.projeto_transparencia
            a = projeto_transparencia.insert_one(t).inserted_id
            lista.append(linha)
            '''json_pause['Pagina'] = pagina
            json_pause['Estado'] = estado
            json_pause['Municipio'] = municipio
            json_pause['Nome'] = nome
            json_pause['Link'] = person_url 
            with open('teste_acrelandia.json', 'r') as arquivos:
                a = json.loads(arquivos.read())
            if linha['Nome'] != a['Nome']:
                with open('teste_acrelandia.json', 'w') as arquivo:
                    #arquivo.write(person_url)
                    json.dump(json_pause, arquivo, indent=5, sort_keys=True)
                projeto_transparencia = banco.projeto_transparencia
                a = projeto_transparencia.insert_one(t).inserted_id
                lista.append(linha)
            else:
                print ('Já salvo no bd')
                #lista.append(linha) '''
        else:
            print ('Já salvo no bd')
    
    return lista
                       
#read_city_page('http://www.portaldatransparencia.gov.br/PortalTransparenciaPesquisaAcaoFavorecido.asp?Exercicio=2016&textoPesquisa=&textoPesquisaAcao=&codigoAcao=8442&codigoFuncao=08&siglaEstado=AC&codigoMunicipio=0643&Pagina=1')

In [ ]:
def read_state_page(state_page_url,estado):
    #print (state_page_url)
    state_page = urllib.request.urlopen(state_page_url)
    state_page_soup = BeautifulSoup(state_page, "lxml")
    b = state_page_soup.findAll("td", {"class":"firstChild"})
    with open('teste_acrelandia.txt', 'r') as arquivo:
        d = arquivo.read()
    print ("Munícipios já lidos \n" + d)
    for td in state_page_soup.findAll("td", {"class":"firstChild"}):
        if td.a is not None:
            #h = verifica_municipio(estado, td.a.getText())
            #if h["Lido_Munia =cipio"] != td.a.getText():
                #print (h)
            
            if td.a.getText() not in d:
                print ("\nESTOU AQUI "+td.a.getText())
                city_url = base_url + td.a.get('href')
                print ('SOU O  city_url\n'+ city_url)
                read_city(city_url)
                c = d  + td.a.getText() + "\n"
                with open('teste_acrelandia.txt', 'w') as arquivo:
                    arquivo.write(c)
    
                
                
                
                
            
def read_state(state_url,estado):
    print ("SOU O state_url\n" + state_url)
    state_page = urllib.request.urlopen(state_url)
    state_page_soup = BeautifulSoup(state_page, "lxml")
    state_pagination = state_page_soup.find(id='paginacao').p.getText()
    print("SOU O state_pagination\n"+state_pagination)
    state_pages = str.split(state_pagination, '/')[1]
    print ("SOU O state_pages\n"+state_pages)
    #state_pages = 1
    for state_page in range(1,int(state_pages)+1):
        state_page_url = state_url + '&Pagina=' + str(state_page)
        print ("SOU O state_page_url\n"+ state_page_url)
        
        read_state_page(state_page_url,estado)

In [ ]:
main_url = 'http://www.portaldatransparencia.gov.br/PortalTransparenciaPesquisaAcaoUF.asp?codigoAcao=8442&codigoFuncao=08&NomeAcao=Transfer%EAncia+de+Renda+Diretamente+%E0s+Fam%EDlias+em+Condi%E7%E3o+de+Pobreza+e+Extrema+Pobreza+%28Lei+n%BA+10%2E836%2C+de+2004%29&Exercicio=2016'
def read_main_page(main_url):
    #print (main_page_url+" ESTOU AQUI")
    main_page = urllib.request.urlopen(main_url)
    main_page_soup = BeautifulSoup(main_page, "lxml")
    a = main_page_soup.findAll("td", {"class":"firstChild"})
    print(len(a))
    for td in main_page_soup.findAll("td", {"class":"firstChild"}):
        if td.a is not None:
            print (td.a.getText())#printa o estado
            state_url = base_url + td.a.get('href')
            #print(state_url)
            read_state(state_url,td.a.getText())

def read_main(main_url):
    main_page = urllib.request.urlopen(main_url)
    soup = BeautifulSoup(main_page, "lxml")
    pagination = soup.find(id='paginacao').p.getText()
    print(pagination)  
    pages = str.split(pagination, '/')[1]
    print (pages)
    for main_page in range(1,int(pages)+1):
        main_page_url = main_url + '&Pagina=' + str(main_page)
        print('eu sou o main_page_url \n'+main_page_url)
        read_main_page(main_page_url)
        

read_main(main_url)
